### 학습 목표

- 초대용량 데이터는 어떻게 처리할까요?


- 데이터를 분할하여 불러오고 처리하는 방법에 대해서 연습해봅니다.


- 불량 검출 문제에 대해 살펴봅니다.


- Class imbalance 문제를 해결하는 방법에 대해 알아봅니다.

## Bosch Production Line Performance

- 공정과정 데이터를 통해 제품 내부 불량 검출하기

- 주어진 공정변수를 이용하여 정상/불량 분류하기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train_cat_part = pd.read_csv("../input/bosch-production-line-performance/train_categorical.csv.zip",
                            nrows=1000)
train_numeric_part = pd.read_csv("../input/bosch-production-line-performance/train_numeric.csv.zip",
                                nrows=1000)
train_date_part = pd.read_csv("../input/bosch-production-line-performance/train_date.csv.zip",
                             nrows=1000)
print(train_cat_part.shape, train_numeric_part.shape, train_date_part.shape)

#### 대용량 데이터가 메인 메모리에 한번에 적재가 안될때!

1. 사용할 column을 몇개 불러온다. (usecols)

2. 데이터를 쪼개서 임시로 불러와서 계산하고, 다시 불러온다. (chunksize)

In [ ]:
train_date_part # start_station, end_station

### train_date 데이터를 요약해봅시다.

- 각 row마다 측정이 처음된 station 번호와, 마지막으로 측정이 된 station 번호를 찾습니다.

In [ ]:
# 1. station별 column 찾기 : 1157개 -> 52개
counts = train_date_part.count() # column이 index로 들어가 있는 Series를 만들려고.
date_cols = counts.reset_index()["index"].str.split("_", expand=True)
col_idx = date_cols.drop_duplicates(1).index  # column 1을 기준으로 중복 제거
date_cols = train_date_part.columns[col_idx]
date_cols

In [ ]:
# 2. start_station, end_station 찾기

train_date = pd.read_csv("../input/bosch-production-line-performance/train_date.csv.zip",
                        usecols=date_cols)

## 전체 데이터에서 column 순서대로 scan하면서 처음으로 NaN이 아닌 column의 station이 start_station,
## 마지막까지 scan하면서 마지막으로 NaN이 아니었던 column의 station이 end_station.
train_date["start_station"] = -1
train_date["end_station"] = -1

for col in train_date.drop(columns=["Id", "start_station", "end_station"]).columns:
    notnulls = ~train_date[col].isnull() # null이 아닌 row들의 위치
    station_name = int(col.split("_")[1][1:])
    
    train_date.loc[(notnulls) & (train_date.start_station == -1), "start_station"] = station_name
    train_date.loc[(notnulls), "end_station"] = station_name
    
train_date

In [ ]:
train_date = train_date[["Id", "start_station", "end_station"]]
train_date

### 차원의 저주(Curse of Dimensionality)

- 머신러닝 모델이 데이터가 존재하는 공간이 너무 고차원일 때, <=> feature(column)가 너무 많을 때

- 머신러닝 모델이 예측력을 잃어버리는 문제. (성능이 저하)


- 해결책

> 차원을 줄입니다! (= column수를 줄인다)

> 1. feature selection : 분석에 사용할 column을 직접 고른다.
>> 결측치가 많은 column을 버리거나, 도메인 지식으로 필요한 변수를 고르거나, ....

> 2. feature extraction : dimensionality reduction method(e.g. PCA)를 사용한다.

In [ ]:
missing_ratio = pd.Series(index=train_numeric_part.columns,
                         data=np.zeros(len(train_numeric_part.columns))) # 970
column_means = pd.Series(index=train_numeric_part.columns,
                         data=np.zeros(len(train_numeric_part.columns)))
length = 0

for chunk in pd.read_csv("../input/bosch-production-line-performance/train_numeric.csv.zip",
                         chunksize=100000):
    #display(chunk.isnull().sum())
    temp = chunk.isnull().sum()  # chunk별 결측치 합
    temp2 = chunk.sum()
    
    length = length + len(chunk)
    missing_ratio = missing_ratio + temp  # 결측치 누적합
    column_means = column_means + temp2  # 실제 값의 누적합
    #display(missing_ratio + chunk.isnull().sum())
    #break

display(missing_ratio) # 결측치 개수
display(column_means / length)

In [ ]:
missing_ratio = missing_ratio / length  # 비율로 변환

In [ ]:
# 결측 비율이 50%를 넘지 않는 column들만 뽑아오겠다.
usecols = train_numeric_part.columns[missing_ratio <= 0.5]
usecols

### 1. 대용량 데이터 다루기

Technique1. 잘라서 불러오기


Technique2. Reducing Memory Technique (dtype 변경하기)

In [ ]:
n_components = 15

from sklearn.decomposition import PCA
data = pd.read_csv("../input/bosch-production-line-performance/train_numeric.csv.zip",
                        usecols=usecols)
data = data.fillna(0) # 결측치를 0으로 채움
ids = data.Id
responses = data.Response
data = data.drop(columns=["Id", "Response"])
pca = PCA(n_components=n_components)
X = pca.fit_transform(data)
pca_df = pd.DataFrame(columns=[f"PC{i}" for i in range(1, n_components+1)],
                     data=X)
display(pca_df)

### 데이터 병합

- 학습에 사용할 데이터로 합쳐줍니다.

- Numeric Data는 PCA로, Date Data는 start_station, end_station 정보를 사용했습니다.

In [ ]:
## 데이터를 합쳐줍니다.
# pca_df (numeric 데이터로 만든 데이터) + train_date(date 데이터로 만든 데이터)
pca_df["Id"] = ids
pca_df["Response"] = responses
X = pd.merge(train_date, pca_df, on="Id")
X

### if, ML

In [ ]:
from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import matthews_corrcoef


y = X.Response
X = X.drop(columns=["Id", "Response"])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0xC0FFEE,
                                                 stratify=y)

clf = LGBMClassifier(max_depth=8, n_estimators=200, random_state=0xC0FFEE, verbose=1)
clf.fit(X_train, y_train)

pred = clf.predict(X_train)
pred2 = clf.predict(X_val)

print("Train MCC : %.4f" % matthews_corrcoef(y_train, pred))
print("Validation MCC : %.4f" % matthews_corrcoef(y_val, pred2))

In [ ]:
import pickle

# 사용한 feature vector 저장
with open("train_data.pk", "wb") as f:
    pickle.dump(X, f)
    
# 학습한 LightGBM 모델 저장
with open("LGBM_model.pk", "wb") as f:
    pickle.dump(clf, f)

### Prediction

- Test data를 불러와서, "동일한" 전처리를 수행합니다.

- 동일한 column(=feature)로 구성된 test data를 학습시킨 모델로 inference한 결과를 제출합니다.

In [ ]:
# test를 하기전에 이미 만들었던 데이터들은 메모리 최적화를 위해 지워줍니다.
import gc

del X
del train_date
del train_cat_part
del train_date_part
del train_numeric_part
gc.collect()

In [ ]:
# 1. test data Load
test_cat_part = pd.read_csv("../input/bosch-production-line-performance/test_categorical.csv.zip",
                            nrows=1000)
test_numeric_part = pd.read_csv("../input/bosch-production-line-performance/test_numeric.csv.zip",
                                nrows=1000)
test_date_part = pd.read_csv("../input/bosch-production-line-performance/test_date.csv.zip",
                             nrows=1000)
print(test_cat_part.shape, test_numeric_part.shape, test_date_part.shape)

In [ ]:
# 2. preprocessing
# 2-1. test_date_part
counts = test_date_part.count() # column이 index로 들어가 있는 Series를 만들려고.
date_cols = counts.reset_index()["index"].str.split("_", expand=True)
col_idx = date_cols.drop_duplicates(1).index  # column 1을 기준으로 중복 제거
date_cols = test_date_part.columns[col_idx]


# 2. start_station, end_station 찾기

test_date = pd.read_csv("../input/bosch-production-line-performance/test_date.csv.zip",
                        usecols=date_cols)

## 전체 데이터에서 column 순서대로 scan하면서 처음으로 NaN이 아닌 column의 station이 start_station,
## 마지막까지 scan하면서 마지막으로 NaN이 아니었던 column의 station이 end_station.
test_date["start_station"] = -1
test_date["end_station"] = -1

for col in test_date.drop(columns=["Id", "start_station", "end_station"]).columns:
    notnulls = ~test_date[col].isnull() # null이 아닌 row들의 위치
    station_name = int(col.split("_")[1][1:])
    
    test_date.loc[(notnulls) & (test_date.start_station == -1), "start_station"] = station_name
    test_date.loc[(notnulls), "end_station"] = station_name
    
display(test_date)
test_date = test_date[["Id", "start_station", "end_station"]]

In [ ]:
# 2-2. Numeric data에서 missing_value 지우고 PCA
# missing_columns는 이미 train에서 구했던걸 사용합니다.
# PCA도 이미 사용했던 걸 사용합니다.
usecols = usecols.drop("Response") # Response column이 없음.

data = pd.read_csv("../input/bosch-production-line-performance/test_numeric.csv.zip",
                        usecols=usecols)
data = data.fillna(0) # 결측치를 0으로 채움
ids = data.Id
data = data.drop(columns=["Id"])
X = pca.transform(data) # fit_transform이 아니라 transform 함수.
pca_df = pd.DataFrame(columns=[f"PC{i}" for i in range(1, n_components+1)],
                     data=X)
display(pca_df)

In [ ]:
# 2-3. 데이터 병합
# pca_df (numeric 데이터로 만든 데이터) + test_date(date 데이터로 만든 데이터)
pca_df["Id"] = ids
X = pd.merge(test_date, pca_df, on="Id")
X

### Test data Inference

In [ ]:
X_test = X.drop(columns=["Id"])
result = clf.predict(X_test)
result

In [ ]:
# make submission
submission = pd.read_csv("../input/bosch-production-line-performance/sample_submission.csv.zip")
submission

In [ ]:
submission["Response"] = result
submission.to_csv("submission.csv", index=False)